In [ ]:
import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype
import xgboost as xgb
import matplotlib
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
# import missingno as msno
import pickle
from sklearn import preprocessing
from sklearn.metrics import roc_curve
from scipy import stats
from scipy.stats import zscore
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold as SKF
from sklearn import metrics
# from fancyimpute import *
from sklearn.metrics.pairwise import cosine_similarity

%matplotlib inline

In [ ]:
def save_obj(obj, name ):
    with open('obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)
    
def display_all(df):
    with pd.option_context("display.max_rows", 1000):
        with pd.option_context("display.max_columns", 1000):
            display(df)

def batch_save(train_x, train_y, valid_x, valid_y, test, postfix):
    train_x.reset_index().to_feather("tmp/train_x_{}".format(postfix))
    train_y.reset_index().to_feather("tmp/train_y_{}".format(postfix))
    valid_x.reset_index().to_feather("tmp/valid_x_{}".format(postfix))
    valid_y.reset_index().to_feather("tmp/valid_y_{}".format(postfix))
    test.reset_index().to_feather("tmp/test_{}".format(postfix))
    
def batch_load(postfix):
    train_x = pd.read_feather("tmp/train_x_{}".format(postfix))
    train_y = pd.read_feather("tmp/train_y_{}".format(postfix))
    valid_x = pd.read_feather("tmp/valid_x_{}".format(postfix))
    valid_y = pd.read_feather("tmp/valid_y_{}".format(postfix))
    return train_x, train_y, valid_x, valid_y

def my_roc(y_true, y_prob):
    if isinstance(y_true,pd.core.series.Series):
        y_true = np.array(y_true.tolist())
    if isinstance(y_true,list):
        y_true = np.array(y_true)
    sort_index = np.argsort(y_prob)[::-1]
    y_prob = y_prob[sort_index]
    y_true = y_true[sort_index]
    num_p = y_true.sum()
    num_n = len(y_true) - num_p
    fp = 0
    tp = 0
    fps = []
    tps = []
    prob_prev = -99
    i = 0
    while i < len(y_true):
        if y_prob[i]!=prob_prev:
            fps.append(fp/num_n)
            tps.append(tp/num_p)
            prob_prev=y_prob[i]
        if y_true[i]==1:
            tp+=1
        else:
            fp+=1
        i+=1
    fps.append(fp/num_n)
    tps.append(tp/num_p)
    return np.array(fps), np.array(tps)

def my_score3(predictions, xtrain): ##Adapted from SKlearn, conservative (actual should be higher)
    ground_truth = xtrain.get_label()
    fpr,tpr = my_roc(ground_truth, predictions)
#     plt.scatter(fpr, tpr)
#     plt.show()
    tpr1 = tpr[(fpr>=0.001).argmax()-1]
    tpr2 = tpr[(fpr>=0.005).argmax()-1] 
    tpr3 = tpr[(fpr>=0.01).argmax()-1]
    return 'score', 0.4 * tpr1 + 0.3 * tpr2 + 0.3* tpr3

def get_ratio(predictions, xtrain):
    ratio_predict = (predictions>0.5).sum()/predictions.shape[0]*100
    # ratio_true = xtrain.get_label().sum()/xtrain.get_label().shape[0]*100
    return 'score', ratio_predict

def norm_standardize(df, start=0):
    for col in df.columns[start:]:
#         avg = df[col].mean()
#         std = df[col].std(ddof=0)
#         if std != 0:
#             df[col] = (df[col]-avg)/std
#         else:
#             print(col)
        a = df[col]
        z = a
        z[~np.isnan(a)] = zscore(a[~np.isnan(a)])
        df[col] = z
            
def norm_maxmin(df, start=0):
    for col in df.columns[start:]:
        df[col]=(df[col]-df[col].min())/(df[col].max()-df[col].min())
        
def process_dates(df,date):
    attrs = ['Day', 'Dayofweek', 'Is_month_end', 'Is_month_start']
    for attr in attrs:
        df[attr] = getattr(df[date].dt, attr.lower())
    return df

In [ ]:
dtype = load_obj('dict_dtype')

my_dict = load_obj('my_dict')

**** Load the training data and test data ****

In [ ]:
data = pd.read_csv("atec_anti_fraud_train.csv",parse_dates=['date'], dtype = dtype)
test = pd.read_csv("atec_anti_fraud_test_a.csv",parse_dates=['date'], dtype = dtype)

In [ ]:
df_missing_ratio = load_obj('df_missing_ratio')
# display_all(df_missing_ratio)

selected_cols = ['f'+str(item) for item in df_missing_ratio[df_missing_ratio['positive_missing_ratio']<0.1].index.tolist()]
all_nan_cols = ['f'+str(item) for item in df_missing_ratio[df_missing_ratio['positive_missing_ratio']>0.9].index.tolist()]

# # use the columns with no or few missing values
# data = data.drop(all_nan_cols, axis=1)
# test = test.drop(all_nan_cols, axis=1)

**** Perform one hot encoding for the columns with no more than 10 unique values ****

In [ ]:
# one hot encoding for the columns with no more than 10 unique values
for col in data.columns[3:]:
    data_unique = data[col].unique()
    test_unique = test[col].unique()
    if data_unique[~np.isnan(data_unique)].min() == test_unique[~np.isnan(test_unique)].min() and \
    data_unique[~np.isnan(data_unique)].max() == test_unique[~np.isnan(test_unique)].max() and \
    data_unique.shape[0] == test_unique.shape[0] and \
    data_unique.shape[0] <= 10:
        data[col].fillna(-1.0)
        test[col].fillna(-1.0)
        for num in data[col].unique():
            new_col = '{}={}'.format(col, num)
            data[new_col] = data[col].apply(lambda x: np.isnan(x) if np.isnan(num) else x==num)
            test[new_col] = test[col].apply(lambda x: np.isnan(x) if np.isnan(num) else x==num)
            data.drop([col], axis=1)
            test.drop([col], axis=1)

**** Perform normalization ****

In [ ]:
# normalization with maxmin
norm_maxmin(data, 3)
norm_maxmin(test, 2)

In [ ]:
# normalization with zscore
norm_standardize(data, 3)
norm_standardize(test, 2)

**** Sort the training data and remove the unlabeled data ****

In [ ]:
# temporially ignore the rows without labels
data.sort_values('date',inplace=True)
# unlabeled = data[data['label']==-1]

In [ ]:
# with the prediction on the unlabeled training data, set the labels for unlabeled data
data.loc[data['label']==-1,'label'] = pd.Series((pred_xgunlabeled>0.5).astype(int), name='label', index=data.loc[data['label']==-1,'label'].index)

In [ ]:
data = data[data['label']!=-1]
data.reset_index(drop=True, inplace=True)

**** calculate the weight of each row with trained RF model ****

In [ ]:
rf_model = load_obj('train_test_shift')
weights = rf_model.predict_proba(data.fillna(-1).iloc[:,3:].values)[:,1:]

print(weights.shape, data.shape)

In [ ]:
# load the weights from Katrina
weights = load_obj('weights')

weights.weight.values.argsort()

In [ ]:
train = data.iloc[weights.weight.values.argsort()[weights.shape[0]//10-weights.shape[0]:],:]
valid = data.iloc[weights.weight.values.argsort()[:weights.shape[0]//10],:]

In [ ]:
np.count_nonzero(weights)

In [ ]:
weights.shape

In [ ]:
weights = 1./weights - 1

**** data segementation based on missing pattern ****

In [ ]:
train_cate = pd.read_csv("obj/id_cate_train.csv")
test_cate = pd.read_csv("obj/id_cate_test.csv")
data = data.merge(train_cate, how='inner', on='id')
test = test.merge(test_cate, how='inner', on='id')
data = data[data['label']!=-1]

In [ ]:
# weights = load_obj('weights')
# weights = weights.drop('label', axis=1)
# data = data.merge(weights, how='inner', on='id')

# weights = data['weight']
# data = data.drop('weight', axis=1)

In [ ]:
data = process_dates(data,'date')
test = process_dates(test,'date')

In [ ]:
# The way of make groups is based on the category cosine similarity analysis below
segment_cate = [[0, 4, 9, 12, 13, 32, 33], [1, 5], [18, 31], [3, 8, 15, 17, 19, 20, 29, 32],\
 [16, 22, 30], [7, 26], [11, 21, 23, 24, 25, 27],[2, 6, 10, 14, 28]]

In [ ]:
test_res = pd.DataFrame(columns=['id','score'])

Model 1 (8 models in total)

In [ ]:
data_seg = data[data['cate'].isin(segment_cate[0])]
test_seg = test[test['cate'].isin(segment_cate[0])]

data_seg = data_seg.drop(['id', 'cate', 'date'], axis=1)
test_seg_id = test_seg.id
test_seg_id = test_seg_id.reset_index(drop=True)
test_seg = test_seg.drop(['id', 'cate', 'date'], axis=1)

# train = data_seg.iloc[:len(data_seg) * 8 // 10]
# valid = data_seg.iloc[len(data_seg) * 8 // 10:]

train_y = data_seg[['label']]
train_x = data_seg.iloc[:,1:]
# valid_y = valid[['label']]
# valid_x = valid.iloc[:,1:]

# print(train_x.shape, valid_x.shape, train_y.shape, valid_y.shape, test_seg.shape)

xgtrain = xgb.DMatrix(train_x.values, label=train_y.values)
# xgval_1 = xgb.DMatrix(valid_x.iloc[:valid_x.shape[0] // 2,:].values,\
#                       label=valid_y.iloc[:valid_x.shape[0] // 2,:].values)
# xgval_2 = xgb.DMatrix(valid_x.iloc[valid_x.shape[0] // 2:,:].values,\
#                       label=valid_y.iloc[valid_x.shape[0] // 2:,:].values)
# xgval = xgb.DMatrix(valid_x.values, label=valid_y.values)
evallist = [(xgtrain, 'train')]#, (xgval_1, 'val_1'), (xgval_2, 'val_2'), (xgval, 'val')]

# set up the parameters
params = {'max_depth': 5, 'eta': 0.1, 'silent': 1, 'objective': 'binary:logistic'}
params['nthread'] = 4
params['eval_metric'] = ['logloss', 'auc']
params["colsample_bytree "] = 0.5
params["scale_pos_weight"] = 2
num_rounds = 200
early_stopping_rounds = 1000

# set up the random seed for testing
params["seed"] = 666

%time xgb_model = xgb.train(params, xgtrain, num_rounds, evallist,\
    feval=my_score3, early_stopping_rounds=early_stopping_rounds)#, xgb_model = xgb_model)

xgtest = xgb.DMatrix(test_seg.values)
# make predictions
preds = xgb_model.predict(xgtest)#, ntree_limit=xgb_model.best_ntree_limit)
res = pd.concat([test_seg_id, pd.Series(list(preds), name='score')], axis=1)
test_res = test_res.append(res)

test_res.to_csv('pred_part_1.csv',index=False)

xgb_model.save_model('model_log/multi/part1.model')

Model 2 (8 models in total)

In [ ]:
data_seg = data[data['cate'].isin(segment_cate[1])]
test_seg = test[test['cate'].isin(segment_cate[1])]

data_seg = data_seg.drop(['id', 'cate', 'date'], axis=1)
test_seg_id = test_seg.id
test_seg_id = test_seg_id.reset_index(drop=True)
test_seg = test_seg.drop(['id', 'cate', 'date'], axis=1)

train_y = data_seg[['label']]
train_x = data_seg.iloc[:,1:]

xgtrain = xgb.DMatrix(train_x.values, label=train_y.values)
evallist = [(xgtrain, 'train')]#, (xgval_1, 'val_1'), (xgval_2, 'val_2'), (xgval, 'val')]

# set up the parameters
params = {'max_depth': 5, 'eta': 0.1, 'silent': 1, 'objective': 'binary:logistic'}
params['nthread'] = 4
params['eval_metric'] = ['logloss', 'auc']
params["colsample_bytree "] = 0.5
params["scale_pos_weight"] = 2
num_rounds = 200
early_stopping_rounds = 1000

# set up the random seed for testing
params["seed"] = 666

%time xgb_model = xgb.train(params, xgtrain, num_rounds, evallist,\
    feval=my_score3, early_stopping_rounds=early_stopping_rounds)#, xgb_model = xgb_model)

In [ ]:
xgtest = xgb.DMatrix(test_seg.values)
# make predictions
preds = xgb_model.predict(xgtest)#, ntree_limit=xgb_model.best_ntree_limit)
res = pd.concat([test_seg_id, pd.Series(list(preds), name='score')], axis=1)
test_res = test_res.append(res)

test_res.to_csv('pred_part_2.csv',index=False)

xgb_model.save_model('model_log/multi/part2.model')

Model 4 (8 models in total)

In [ ]:
data_seg = data[data['cate'].isin(segment_cate[3])]
test_seg = test[test['cate'].isin(segment_cate[3])]

data_seg = data_seg.drop(['id', 'cate', 'date'], axis=1)
test_seg_id = test_seg.id
test_seg_id = test_seg_id.reset_index(drop=True)
test_seg = test_seg.drop(['id', 'cate', 'date'], axis=1)

train_y = data_seg[['label']]
train_x = data_seg.iloc[:,1:]

xgtrain = xgb.DMatrix(train_x.values, label=train_y.values)
evallist = [(xgtrain, 'train')]#, (xgval_1, 'val_1'), (xgval_2, 'val_2'), (xgval, 'val')]

# set up the parameters
params = {'max_depth': 5, 'eta': 0.1, 'silent': 1, 'objective': 'binary:logistic'}
params['nthread'] = 4
params['eval_metric'] = ['logloss', 'auc']
params["colsample_bytree "] = 0.5
params["scale_pos_weight"] = 2
num_rounds = 180
early_stopping_rounds = 1000

# set up the random seed for testing
params["seed"] = 666

%time xgb_model = xgb.train(params, xgtrain, num_rounds, evallist,\
    feval=my_score3, early_stopping_rounds=early_stopping_rounds)#, xgb_model = xgb_model)

xgtest = xgb.DMatrix(test_seg.values)
# make predictions
preds = xgb_model.predict(xgtest)#, ntree_limit=xgb_model.best_ntree_limit)
res = pd.concat([test_seg_id, pd.Series(list(preds), name='score')], axis=1)
test_res = test_res.append(res)
test_res.to_csv('pred_part_4.csv',index=False)
xgb_model.save_model('model_log/multi/part4.model')

Model 6 (8 models in total)

In [ ]:
data_seg = data[data['cate'].isin(segment_cate[5])]
test_seg = test[test['cate'].isin(segment_cate[5])]

data_seg = data_seg.drop(['id', 'cate', 'date'], axis=1)
test_seg_id = test_seg.id
test_seg_id = test_seg_id.reset_index(drop=True)
test_seg = test_seg.drop(['id', 'cate', 'date'], axis=1)

train_y = data_seg[['label']]
train_x = data_seg.iloc[:,1:]

xgtrain = xgb.DMatrix(train_x.values, label=train_y.values)
evallist = [(xgtrain, 'train')]#, (xgval_1, 'val_1'), (xgval_2, 'val_2'), (xgval, 'val')]

# set up the parameters
params = {'max_depth': 5, 'eta': 0.3, 'silent': 1, 'objective': 'binary:logistic'}
params['nthread'] = 4
params['eval_metric'] = ['logloss', 'auc']
params["colsample_bytree "] = 0.5
params["scale_pos_weight"] = 2
num_rounds = 500
early_stopping_rounds = 1000

# set up the random seed for testing
params["seed"] = 666

%time xgb_model = xgb.train(params, xgtrain, num_rounds, evallist,\
    feval=my_score3, early_stopping_rounds=early_stopping_rounds)#, xgb_model = xgb_model)

xgtest = xgb.DMatrix(test_seg.values)
# make predictions
preds = xgb_model.predict(xgtest)#, ntree_limit=xgb_model.best_ntree_limit)
res = pd.concat([test_seg_id, pd.Series(list(preds), name='score')], axis=1)
test_res = test_res.append(res)
test_res.to_csv('pred_part_6.csv',index=False)
xgb_model.save_model('model_log/multi/part6.model')

model 6 experiment (train on the model 1)

In [ ]:
data_seg = data[data['cate'].isin(segment_cate[5])]
test_seg = test[test['cate'].isin(segment_cate[5])]

data_seg = data_seg.drop(['id', 'cate', 'date'], axis=1)
test_seg_id = test_seg.id
test_seg_id = test_seg_id.reset_index(drop=True)
test_seg = test_seg.drop(['id', 'cate', 'date'], axis=1)

train = data_seg.iloc[:len(data_seg) * 8 // 10]
valid = data_seg.iloc[len(data_seg) * 8 // 10:]

train_y = train[['label']]
train_x = train.iloc[:,1:]
valid_y = valid[['label']]
valid_x = valid.iloc[:,1:]

# print(train_x.shape, valid_x.shape, train_y.shape, valid_y.shape, test_seg.shape)

xgtrain = xgb.DMatrix(train_x.values, label=train_y.values)
xgval_1 = xgb.DMatrix(valid_x.iloc[:valid_x.shape[0] // 2,:].values,\
                      label=valid_y.iloc[:valid_x.shape[0] // 2,:].values)
xgval_2 = xgb.DMatrix(valid_x.iloc[valid_x.shape[0] // 2:,:].values,\
                      label=valid_y.iloc[valid_x.shape[0] // 2:,:].values)
xgval = xgb.DMatrix(valid_x.values, label=valid_y.values)
evallist = [(xgtrain, 'train'), (xgval_1, 'val_1'), (xgval_2, 'val_2'), (xgval, 'val')]

# set up the parameters
params = {'max_depth': 5, 'eta': 0.3, 'silent': 1, 'objective': 'binary:logistic'}
params['nthread'] = 4
params['eval_metric'] = ['logloss', 'auc']
params["colsample_bytree "] = 0.5
params["scale_pos_weight"] = 2
num_rounds = 200
early_stopping_rounds = 1000

# set up the random seed for testing
params["seed"] = 666

xgb_model = xgb.Booster({'nthread': 4})  # init model
xgb_model.load_model('model_log/multi/part1.model')  # load model

%time xgb_model = xgb.train(params, xgtrain, num_rounds, evallist,\
    feval=my_score3, early_stopping_rounds=early_stopping_rounds, xgb_model = xgb_model)

Model 8 (8 models in total)

In [ ]:
data_seg = data[data['cate'].isin(segment_cate[7])]
test_seg = test[test['cate'].isin(segment_cate[7])]

data_seg = data_seg.drop(['id', 'cate', 'date'], axis=1)
test_seg_id = test_seg.id
test_seg_id = test_seg_id.reset_index(drop=True)
test_seg = test_seg.drop(['id', 'cate', 'date'], axis=1)

train_y = data_seg[['label']]
train_x = data_seg.iloc[:,1:]

xgtrain = xgb.DMatrix(train_x.values, label=train_y.values)
evallist = [(xgtrain, 'train')]#, (xgval_1, 'val_1'), (xgval_2, 'val_2'), (xgval, 'val')]

# set up the parameters
params = {'max_depth': 5, 'eta': 0.3, 'silent': 1, 'objective': 'binary:logistic'}
params['nthread'] = 4
params['eval_metric'] = ['logloss', 'auc']
params["colsample_bytree "] = 0.5
params["scale_pos_weight"] = 2
num_rounds = 300
early_stopping_rounds = 1000

# set up the random seed for testing
params["seed"] = 666

%time xgb_model = xgb.train(params, xgtrain, num_rounds, evallist,\
    feval=my_score3, early_stopping_rounds=early_stopping_rounds)#, xgb_model = xgb_model)

xgtest = xgb.DMatrix(test_seg.values)
# make predictions
preds = xgb_model.predict(xgtest)#, ntree_limit=xgb_model.best_ntree_limit)
res = pd.concat([test_seg_id, pd.Series(list(preds), name='score')], axis=1)
test_res = test_res.append(res)
test_res.to_csv('pred_part_8.csv',index=False)
xgb_model.save_model('model_log/multi/part8.model')

Model 3,5,7 (8 models in total)

In [ ]:
data_seg = data[data['cate'].isin(segment_cate[2]+segment_cate[4]+segment_cate[6])]
test_seg = test[test['cate'].isin(segment_cate[2]+segment_cate[4]+segment_cate[6])]

data_seg = data_seg.drop(['id', 'cate', 'date'], axis=1)
test_seg_id = test_seg.id
test_seg_id = test_seg_id.reset_index(drop=True)
test_seg = test_seg.drop(['id', 'cate', 'date'], axis=1)

train_y = data_seg[['label']]
train_x = data_seg.iloc[:,1:]

xgtrain = xgb.DMatrix(train_x.values, label=train_y.values)
evallist = [(xgtrain, 'train')]#, (xgval_1, 'val_1'), (xgval_2, 'val_2'), (xgval, 'val')]

# set up the parameters
params = {'max_depth': 3, 'eta': 0.2, 'silent': 1, 'objective': 'binary:logistic'}
params['nthread'] = 4
params['eval_metric'] = ['logloss', 'auc']
params["colsample_bytree "] = 0.5
params["scale_pos_weight"] = 2
num_rounds = 60
early_stopping_rounds = 1000

# set up the random seed for testing
params["seed"] = 666

xgb_model = xgb.Booster({'nthread': 4})  # init model
xgb_model.load_model('model_log/multi/part1.model')  # load model

%time xgb_model = xgb.train(params, xgtrain, num_rounds, evallist,\
    feval=my_score3, early_stopping_rounds=early_stopping_rounds, xgb_model = xgb_model)

xgtest = xgb.DMatrix(test_seg.values)
# make predictions
preds = xgb_model.predict(xgtest)#, ntree_limit=xgb_model.best_ntree_limit)
res = pd.concat([test_seg_id, pd.Series(list(preds), name='score')], axis=1)
test_res = test_res.append(res)
test_res.to_csv('pred_part_final.csv',index=False)
xgb_model.save_model('model_log/multi/part357.model')

Put all predictions together

In [ ]:
test_res=pd.read_csv('pred_part_final.csv')
test_res=test_res.drop_duplicates(subset=['id'], keep='last')
test_res = pd.DataFrame(test.id).merge(test_res,how='inner',on='id',indicator=True)
test_res = test_res.drop('_merge',axis=1)
test_res.to_csv('0021.csv', index=False)

**** find the similarities between categories, put similar categories together ****

In [ ]:
cate_mapping = list()
for i in range(34):
    temp = data[data['cate']==i].head(1)\
        .drop(['id','label','date', 'cate', 'Day', 'Dayofweek', 'Is_month_end', 'Is_month_start'], axis=1).values[0]
    temp_list = list()
    for j in range(297):
        if np.isnan(temp[j]):
            temp_list.append(0)
        else:
            temp_list.append(1)
    cate_mapping.append(tuple(temp_list))

for i in range(34):
    print('=========================')
    temp_list = list()
    for j in range(34):
        if j==i:
            temp_list.append(0)
        else:
            temp_list.append(cosine_similarity([list(cate_mapping[i])], [list(cate_mapping[j])])[0][0])
    print(i, temp_list)
    print(np.argmax(temp_list), temp_list[np.argmax(temp_list)])
    print(np.array(temp_list).argsort()[32], temp_list[np.array(temp_list).argsort()[32]])
    print(np.array(temp_list).argsort()[31], temp_list[np.array(temp_list).argsort()[31]])
    print(np.array(temp_list).argsort()[30], temp_list[np.array(temp_list).argsort()[30]])
        

j = 0
for i in range(34):
    print('++++++++++++++++    {}    ++++++++++++++++'.format(i))
    print(cate_mapping[i])

**** incrementally train model ****

In [ ]:
num_fraction = 10
fraction_size = data.shape[0]//10

x_batches = []
y_batches = []
unlabeled = []
for i in range(num_fraction):
    if i!=num_fraction-1:
        data_portion = data.iloc[i*fraction_size:(i+1)*fraction_size,:]
    else:
        data_portion = data.iloc[i*fraction_size:,:]
    unlabeled.append(data_portion[data_portion['label']==-1].iloc[:,3:])
    x_batches.append(data_portion[data_portion['label']!=-1].iloc[:,3:])
    y_batches.append(data_portion[data_portion['label']!=-1]['label'])

In [ ]:
# initial model
# set up the parameters
params = {'max_depth': 8, 'eta': 0.1, 'silent': 1, 'objective': 'binary:logistic'}
params['nthread'] = 4
params['eval_metric'] = ['error', 'auc']
params["scale_pos_weight"] = 5
num_rounds = 50
early_stopping_rounds = 20

# set up the random seed for testing
params["seed"] = 10

xgtrain = xgb.DMatrix(x_batches[0].values, label=y_batches[0].values)
xgb_model = xgb.train(params, xgtrain,\
                      num_rounds,\
                      [(xgtrain, 'current'),\
                       (xgb.DMatrix(x_batches[1].values, label=y_batches[1].values), 'next')],\
                      # feval=get_ratio,\
                      early_stopping_rounds=early_stopping_rounds)

In [ ]:
# predict the unlabeled data of the first fraction
pred_unlabeled = xgb_model.predict(xgb.DMatrix(unlabeled[0].values))
unlabeled[0]['label'] = pd.Series((pred_unlabeled>0.5).astype(int), name='label',\
                                      index = unlabeled[0].index)

In [ ]:
# incrementally train the model
for fraction_id in range(1,num_fraction):
    print('=========================================================================')
    print('=====       '+str(num_fraction)+' fractions in total, training on the fraction {}'.format(fraction_id+1)+'       =====')
    print('=========================================================================')
#     xgtrain = xgb.DMatrix(x_batches[fraction_id].append(unlabeled[fraction_id-1].iloc[:,:-1]).values,\
#                           label=y_batches[fraction_id].append(unlabeled[fraction_id-1]['label']).values)
    xgtrain = xgb.DMatrix(x_batches[fraction_id].values,\
                          label=y_batches[fraction_id].values)
    if fraction_id != num_fraction-1:
        xgb_model = xgb.train(params, xgtrain,\
            num_rounds,\
            [(xgtrain, 'current'),\
            (xgb.DMatrix(x_batches[fraction_id+1].values, label=y_batches[fraction_id+1].values), 'next')],\
            # feval=my_score3,\
            early_stopping_rounds=early_stopping_rounds,\
            xgb_model = xgb_model)
#         pred_unlabeled = xgb_model.predict(xgb.DMatrix(unlabeled[fraction_id].values))
#         unlabeled[fraction_id]['label'] = pd.Series((pred_unlabeled>0.5).astype(int), name='label',\
#                                                     index = unlabeled[fraction_id].index)
    else:
        num_rounds = 50
        xgb_model = xgb.train(params, xgtrain,\
            num_rounds,\
            [(xgtrain, 'current')],\
            # feval=my_score3,\
            early_stopping_rounds=early_stopping_rounds,\
            xgb_model = xgb_model,)

**** Create validation set and training set ****

In [ ]:
train = data.iloc[:len(data) * 8 // 10]
valid = data.iloc[len(data) * 8 // 10:]

In [ ]:
train_weights = weights.weight.values[:len(data) * 8 // 10]
train_weights /= np.mean(train_weights) # Normalizing the weights

In [ ]:
train_y = train[['label']]
train_x = train.iloc[:,3:]
valid_y = valid[['label']]
valid_x = valid.iloc[:,3:]

In [ ]:
xgtrain = xgb.DMatrix(train_x.values, weight=train_weights, label=train_y.values)
xgval_1 = xgb.DMatrix(valid_x.iloc[:valid_x.shape[0] // 2,:].values,\
                      label=valid_y.iloc[:valid_x.shape[0] // 2,:].values)
xgval_2 = xgb.DMatrix(valid_x.iloc[valid_x.shape[0] // 2:,:].values,\
                      label=valid_y.iloc[valid_x.shape[0] // 2:,:].values)
xgval = xgb.DMatrix(valid_x.values,\
                      label=valid_y.iloc[:,:].values)
evallist = [(xgtrain, 'train'), (xgval_1, 'val_1'), (xgval_2, 'val_2'), (xgval, 'val')]

In [ ]:
# normal xgtrain and xgval
xgtrain = xgb.DMatrix(train_x.values, label=train_y.values)
xgval_1 = xgb.DMatrix(valid_x.iloc[:valid_x.shape[0] // 2,:].values,\
                      label=valid_y.iloc[:valid_x.shape[0] // 2,:].values)
xgval_2 = xgb.DMatrix(valid_x.iloc[valid_x.shape[0] // 2:,:].values,\
                      label=valid_y.iloc[valid_x.shape[0] // 2:,:].values)
xgval = xgb.DMatrix(valid_x.values, label=valid_y.iloc[:,:].values)
evallist = [(xgtrain, 'train'), (xgval_1, 'val_1'), (xgval_2, 'val_2'), (xgval, 'val')]

**** Train the model ****

In [ ]:
# set up the parameters
params = {'max_depth': 6, 'eta': 0.1, 'silent': 1, 'objective': 'binary:logistic'}
params['nthread'] = 4
params['eval_metric'] = ['logloss', 'auc']
params["colsample_bytree "] = 0.5
params["scale_pos_weight"] = 2
num_rounds = 300
early_stopping_rounds = 1000

# set up the random seed for testing
#params["seed"] = 8

In [ ]:
%time xgb_model = xgb.train(params, xgtrain, num_rounds, evallist,\
    feval=my_score3, early_stopping_rounds=early_stopping_rounds)#, xgb_model = xgb_model)

**** save or load the model ****

In [ ]:
# save the model
xgb_model.save_model('model_log/0020.model')
# dump model with feature map
xgb_model.dump_model('model_log/dumpraw0020.txt')

In [ ]:
# load the model
xgb_model = xgb.Booster({'nthread': 4})  # init model
xgb_model.load_model('model_log/0020.model')  # load model

**** train the existing model on the validation set ****

In [ ]:
xgval = xgb.DMatrix(valid_x.values, label=valid_y.values)

In [ ]:
%time xgb_model = xgb.train(params, xgval, num_rounds, [(xgval, 'validation')], feval=my_score3, early_stopping_rounds=early_stopping_rounds, xgb_model = xgb_model)

**** predict on the testset ****

In [ ]:
# load test data
test_whole = pd.read_feather("tmp/test_native")

In [ ]:
xgtest = xgb.DMatrix(test.iloc[:,2:].values)

# make predictions
preds = xgb_model.predict(xgtest)#, ntree_limit=xgb_model.best_ntree_limit)

res = pd.concat([test.id, pd.Series(list(preds), name='score')], axis=1)

res.to_csv("submission/0020.csv", index=False)

In [ ]:
# the positive ratio of the test data
print('Ratio of positive label in unlabeled data: {}%'.format((preds>0.5).sum()/preds.shape[0]*100))

**** predict on the unlabeled training set ****

In [ ]:
xgunlabeled = xgb.DMatrix(unlabeled.iloc[:,3:].values)

In [ ]:
unlabeled.reset_index(drop=True, inplace=True)

In [ ]:
pred_xgunlabeled = xgb_model.predict(xgunlabeled)

In [ ]:
# set the labels for the unlabeled training data
unlabeled['label'] = pd.Series((pred_xgunlabeled>0.5).astype(int), name='label')

In [ ]:
res = pd.concat([unlabeled.id, pd.Series(list(pred_xgunlabeled), name='score')], axis=1)
res.to_csv("Yabin_unlabeled0011.csv", index=False)

In [ ]:
# the positive ratio of the unlabeled data
print('Ratio of positive label in unlabeled data: {}%'.format((pred_xgunlabeled>0.5).sum()/res.shape[0]*100))